In [79]:
import pandas as pd
from youtube_search import YoutubeSearch
from pytubefix import YouTube


In [80]:
data = pd.read_csv('data/data.csv', sep=';')

data.head()

,nama,daerah,keyword,lirik
0,Kicir-Kicir,Jakarta,jakarta,NaN
1,Ondel Ondel,jakarta,Jakarta,NaN
2,Manuk Dadali,jawa barat,jawa barat,NaN


In [81]:
def search_yt(query):
    results = YoutubeSearch(query, max_results=5).to_dict()

    for i in range(5):
        results[i]['url'] = 'https://www.youtube.com' + results[i]['url_suffix']
        
    return results

def dl_video(query):
    yt = YouTube(query, 'MWEB')
    print(f'Downloading {yt.title}...')
    
    ys = yt.streams.get_audio_only()
    ys.download(mp3=True, output_path='songs', filename=yt.title)
    
    return f'/songs/{yt.title}.mp3'
    

In [82]:
results = []

for index, row in data.iterrows():
    songs = search_yt(f"{row['nama']} asal {row['daerah']}")
    
    result_entry = {
        'nama': row['nama'],
        'daerah': row['daerah'],
        'keyword': row['keyword'],
        'songs': []
    }

    for song in songs:
        path = dl_video(song['url'])
        result_entry['songs'].append({
            'title': song['title'],
            'url': song['url'],
            'path': path
        })
    
    results.append(result_entry)
